In [68]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [127]:
stores = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/stores.csv", )
genres = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/genres.csv", )
goods = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/goods.csv", )
train_df = pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/train.csv")
test_df  =pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/test.csv" )

In [128]:
# goods_genre_idを追加
train_df = pd.merge(train_df, goods[['goods_id', 'goods_genre_id']], on='goods_id')
test_df = pd.merge(test_df, goods[['goods_id', 'goods_genre_id']], on='goods_id')


# 月ごとの売り上げ（目的変数）を追加 
train_df['units_sold_month'] = train_df.groupby(['num_month', 'store_id',  'goods_id']).units_sold_day.transform(np.sum)

# 予測変数が月の売り上げ個数なので、同じ月で(store_id, goods_id)が重複している行を削除
train_df = train_df.drop_duplicates(subset=['num_month', 'store_id', 'goods_id']).reset_index(drop=True)

# 使用しない特徴量を削除
train_df.drop(['yy_mm_dd', 'units_sold_day'], axis=1, inplace=True)

# testデータにnum_month（0）を追加
test_df['num_month'] = 0

In [129]:
all_df = pd.concat([train_df, test_df])
#目的変数の shift 特徴量を作成
n = 5
for i in range(n):
    all_df[f'units_sold_month_{i+1}'] = all_df.groupby(['store_id', 'goods_id']).units_sold_month.shift(i+1)

In [130]:
# 統計量の特徴量を作成する関数
def stats_features(df, columns1, columns2):
    for column1 in columns1:
        for column2 in columns2:
            df[f'{column1}_{column2}_mean'] = df[column1].map(train_df.groupby(column1)[column2].mean())
            df[f'{column1}_{column2}_median'] = df[column1].map(train_df.groupby(column1)[column2].median())
            df[f'{column1}_{column2}_std'] = df[column1].map(train_df.groupby(column1)[column2].std())
    
    return df

In [132]:
# 統計量の特徴量を作成
columns1 = ['goods_genre_id', 'store_id', 'goods_id']
columns2 = ['units_sold_month', 'price']

all_df = stats_features(all_df, columns1, columns2)

In [133]:
all_df

,num_month,store_id,goods_id,price,goods_genre_id,units_sold_month,index,units_sold_month_1,units_sold_month_2,units_sold_month_3,...,store_id_units_sold_month_std,store_id_price_mean,store_id_price_median,store_id_price_std,goods_id_units_sold_month_mean,goods_id_units_sold_month_median,goods_id_units_sold_month_std,goods_id_price_mean,goods_id_price_median,goods_id_price_std
0,-33,43,17718,44943.0,80,10.0,NaN,NaN,NaN,NaN,...,8.928907,78687.435326,39532.0,151511.990369,11.890893,7.0,14.535725,94828.707045,79623.0,84456.161230
1,-33,33,17718,99490.0,80,3.0,NaN,NaN,NaN,NaN,...,2.878891,68382.714074,35025.0,129998.585084,11.890893,7.0,14.535725,94828.707045,79623.0,84456.161230
2,-33,36,17718,9790.0,80,3.0,NaN,NaN,NaN,NaN,...,4.158667,85338.739077,40001.0,167440.631807,11.890893,7.0,14.535725,94828.707045,79623.0,84456.161230
3,-33,32,17718,9791.0,80,7.0,NaN,NaN,NaN,NaN,...,11.741852,64077.361196,30133.0,125278.463834,11.890893,7.0,14.535725,94828.707045,79623.0,84456.161230
4,-33,28,17718,49964.0,80,5.0,NaN,NaN,NaN,NaN,...,8.967838,84549.930945,40073.0,154119.066516,11.890893,7.0,14.535725,94828.707045,79623.0,84456.161230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0,56,1877,NaN,20,NaN,192326.0,1.0,1.0,10.0,...,20.551817,62534.913020,30041.0,117655.640237,2.922764,2.0,2.362576,197234.386179,184416.5,65431.363698
214196,0,57,1877,NaN,20,NaN,194744.0,2.0,3.0,1.0,...,4.040918,67010.685146,30138.0,142863.951195,2.922764,2.0,2.362576,197234.386179,184416.5,65431.363698
214197,0,58,1877,NaN,20,NaN,203042.0,2.0,2.0,6.0,...,6.040283,68379.909097,30261.0,138029.693547,2.922764,2.0,2.362576,197234.386179,184416.5,65431.363698
214198,0,59,1877,NaN,20,NaN,207828.0,1.0,2.0,1.0,...,4.033359,74909.003653,35010.5,152445.968393,2.922764,2.0,2.362576,197234.386179,184416.5,65431.363698


In [134]:
# train と test に戻す
train_df = all_df[:1719470]
test_df = all_df[1719470:]

In [135]:
# モデルで学習する形にする
X_df = train_df[train_df['num_month'] > -12].reset_index(drop=True).drop(['price', 'units_sold_month', 'index'], axis=1)
y_df = train_df[train_df['num_month'] > -12].reset_index(drop=True).units_sold_month
X_test_df = test_df.sort_values('index').reset_index(drop=True).drop(['price', 'units_sold_month', 'index'], axis=1)

X = X_df.values
y = y_df.values
X_test = X_test_df #.values

In [136]:
X_test

,num_month,store_id,goods_id,goods_genre_id,units_sold_month_1,units_sold_month_2,units_sold_month_3,units_sold_month_4,units_sold_month_5,goods_genre_id_units_sold_month_mean,...,store_id_units_sold_month_std,store_id_price_mean,store_id_price_median,store_id_price_std,goods_id_units_sold_month_mean,goods_id_units_sold_month_median,goods_id_units_sold_month_std,goods_id_price_mean,goods_id_price_median,goods_id_price_std
0,0,3,15900,70,1.0,2.0,NaN,NaN,NaN,2.323562,...,3.212355,120133.544361,69675.5,204878.773818,1.833333,1.0,2.296816,63488.250000,69662.5,9787.214545
1,0,3,18326,56,NaN,NaN,NaN,NaN,NaN,1.344150,...,3.212355,120133.544361,69675.5,204878.773818,1.122222,1.0,0.390247,20078.848148,19933.0,3178.354880
2,0,3,5665,4,1.0,1.0,2.0,2.0,1.0,2.803762,...,3.212355,120133.544361,69675.5,204878.773818,1.685185,1.0,1.488693,222454.861111,209450.0,43296.628448
3,0,3,15859,71,2.0,NaN,NaN,NaN,NaN,2.271112,...,3.212355,120133.544361,69675.5,204878.773818,1.575342,1.0,1.279299,70958.534247,44117.0,45824.369901
4,0,3,8826,66,NaN,NaN,NaN,NaN,NaN,2.599956,...,3.212355,120133.544361,69675.5,204878.773818,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0,60,8851,41,NaN,NaN,NaN,NaN,NaN,1.748253,...,3.557091,81676.015555,39772.5,167867.864223,1.111111,1.0,0.333333,28738.000000,29690.0,3337.066788
214196,0,60,1539,20,2.0,6.0,NaN,NaN,NaN,2.434062,...,3.557091,81676.015555,39772.5,167867.864223,2.364706,2.0,1.895151,61915.435294,53813.0,19549.728204
214197,0,60,13719,70,2.0,6.0,2.0,NaN,NaN,2.323562,...,3.557091,81676.015555,39772.5,167867.864223,2.024691,1.0,1.724640,20758.098765,22611.0,2833.728717
214198,0,60,6074,23,1.0,1.0,1.0,NaN,NaN,1.665443,...,3.557091,81676.015555,39772.5,167867.864223,1.245370,1.0,0.646980,110807.861111,100732.5,15923.911793


In [138]:
# optuna で求めたパラメータ
params = {
          'objective': 'regression',
          'metric': 'rmse',
          'lambda_l1': 1.731879036922e-06, 
          'lambda_l2': 0.008249579034746, 
          'max_depth': 3, 
          'num_leaves': 10, 
          'feature_fraction': 0.99996999864248, 
          'bagging_fraction': 0.63648252566587, 
          'bagging_freq': 5, 
          'min_child_samples': 7,
          'num_boost_round': 273,
          'random_state': 42}

In [139]:
# 予測
import lightgbm as lgb
lgb_train = lgb.Dataset(X, y)
lgbm = lgb.train(params, lgb_train)
lgb_pred = lgbm.predict(X_test)
lgb_pred = lgb_pred.astype(int)
lgb_pred = np.where(lgb_pred < 0, 0, lgb_pred)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009907 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3036
[LightGBM] [Info] Number of data points in the train set: 467420, number of used features: 26
[LightGBM] [Info] Start training from score 1.990490
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [140]:
# 提出ファイルの作成
submission  =pd.read_csv("C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/sample_submission.csv", index_col=0 )
submission["units_sold_month"] = lgb_pred
submission

,units_sold_month
index,
0,1
1,0
2,1
3,1
4,0
...,...
214195,0
214196,1
214197,1


In [141]:
# 作成したファイルをダウンロード
submission.to_csv('C:/Users/daisu/OneDrive/Desktop/GCI/drive-download-20201224T003704Z-001/late_lgb_cliped.csv',index=False)